<a href="https://colab.research.google.com/github/shafilahamed/CAPSTONE-2-retail-sales-prediction/blob/main/individual_notebook_rossman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Sales Prediction : Predicting sales of a major store chain Rossmann</u></b>

## <b> Problem Description </b>

### Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

### You are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

## <b> Data Description </b>

### <b>Rossmann Stores Data.csv </b> - historical data including Sales
### <b>store.csv </b> - supplemental information about the stores


### <b><u>Data fields</u></b>
### Most of the fields are self-explanatory. The following are descriptions for those that aren't.

* #### Id - an Id that represents a (Store, Date) duple within the test set
* #### Store - a unique Id for each store
* #### Sales - the turnover for any given day (this is what you are predicting)
* #### Customers - the number of customers on a given day
* #### Open - an indicator for whether the store was open: 0 = closed, 1 = open
* #### StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
* #### SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
* #### StoreType - differentiates between 4 different store models: a, b, c, d
* #### Assortment - describes an assortment level: a = basic, b = extra, c = extended
* #### CompetitionDistance - distance in meters to the nearest competitor store
* #### CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
* #### Promo - indicates whether a store is running a promo on that day
* #### Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
* #### Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
* #### PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score as r2, mean_squared_error as mse
import math
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filepath='/content/drive/MyDrive/retail sales regression/'
store_df=pd.read_csv(filepath + 'Copy of store.csv')
rossman_df=pd.read_csv(filepath + 'Copy of Rossmann Stores Data.csv',low_memory=False)

In [ ]:
store_df.head()


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [ ]:
rossman_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [ ]:
rossman_df.shape,store_df.shape

((1017209, 9), (1115, 10))

In [ ]:
store_df.isna().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [ ]:
store_new = store_df.copy()
store_new['CompetitionDistance'] = store_new['CompetitionDistance'].fillna(0)
store_new['CompetitionOpenSinceMonth'] = store_new['CompetitionOpenSinceMonth'].fillna(0)
competition_year_mode = store_new['CompetitionOpenSinceYear'].mode().iloc[0]
store_new['CompetitionOpenSinceYear'] = store_new['CompetitionOpenSinceYear'].fillna(competition_year_mode)


store_new['Promo2SinceWeek'] = store_new['Promo2SinceWeek'].fillna(0)
promo2year_mode = store_new['Promo2SinceYear'].mode().iloc[0]
store_new['Promo2SinceYear'] = store_new['Promo2SinceYear'].fillna(promo2year_mode)
promo_interval_mode = store_new['PromoInterval'].mode().iloc[0]
store_new['PromoInterval'] = store_new['PromoInterval'].fillna(promo_interval_mode)
store_new.isna().sum()

Store                        0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
dtype: int64

In [ ]:
store_new.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,0.0,2011.0,"Jan,Apr,Jul,Oct"
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,0.0,2011.0,"Jan,Apr,Jul,Oct"
4,5,a,a,29910.0,4.0,2015.0,0,0.0,2011.0,"Jan,Apr,Jul,Oct"


In [ ]:
final1 = pd.merge(left=rossman_df, right=store_new,
                               how='left', on=['Store'])
final1.head()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,2011.0,"Jan,Apr,Jul,Oct"
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,0.0,2011.0,"Jan,Apr,Jul,Oct"
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,0.0,2011.0,"Jan,Apr,Jul,Oct"


In [ ]:
final1['Date'] = pd.to_datetime(final1['Date'])
final1['sales_month'] = final1['Date'].dt.month
cols_drop = ['Date', 'Customers', 'Store']
final1['StateHoliday'] = final1['StateHoliday'].replace({0: '0'})
final1 = final1.drop(cols_drop, axis=1)
final1_dummies = pd.get_dummies(final1, drop_first=True)

In [ ]:
final1['Sales'].mean()

5773.818972305593

In [ ]:
final1_dummies.shape

(1017209, 22)

In [ ]:
final1_dummies.head()

,DayOfWeek,Sales,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,sales_month,StateHoliday_a,StateHoliday_b,StateHoliday_c,StoreType_b,StoreType_c,StoreType_d,Assortment_b,Assortment_c,"PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec"
0,5,5263,1,1,1,1270.0,9.0,2008.0,0,0.0,2011.0,7,0,0,0,0,1,0,0,0,1,0
1,5,6064,1,1,1,570.0,11.0,2007.0,1,13.0,2010.0,7,0,0,0,0,0,0,0,0,1,0
2,5,8314,1,1,1,14130.0,12.0,2006.0,1,14.0,2011.0,7,0,0,0,0,0,0,0,0,1,0
3,5,13995,1,1,1,620.0,9.0,2009.0,0,0.0,2011.0,7,0,0,0,0,1,0,0,1,1,0
4,5,4822,1,1,1,29910.0,4.0,2015.0,0,0.0,2011.0,7,0,0,0,0,0,0,0,0,1,0


In [ ]:
dep_var='Sales'
indep_var=final1_dummies.columns.drop(dep_var)

In [ ]:
X=final1_dummies[indep_var]
y=final1_dummies[dep_var]


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((813767,), (203442,), (813767, 21), (203442, 21))

In [ ]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
dt_regressor=DecisionTreeRegressor(max_depth=5)
dt_regressor.fit(X_train, y_train)
y_pred=dt_regressor.predict(X_test)
y_train_pred=dt_regressor.predict(X_train)
print('dt_regressor R^2: ', r2(y_test,y_pred))
print('dt_regressor RMSE: ', math.sqrt(mse(y_test, y_pred)))
print('dt_regressor RMSE: ', math.sqrt(mse(y_train, y_train_pred)))
print('dt_regressor RMSE: ',(np.sqrt(np.mean(np.square((y_test-y_pred)/y_test))))*100

SyntaxError: ignored

In [ ]:
rf_regressor=RandomForestRegressor(n_estimators =500,max_depth=5)
rf_regressor.fit(X_train, y_train)
y_pred=rf_regressor.predict(X_test)
print('rf_regressor R^2: ', r2(y_test,y_pred))
print('rf_regressor RMSE: ', math.sqrt(mse(y_test, y_pred)))

rf_regressor R^2:  0.5871323581837904
rf_regressor RMSE:  2480.17582543292


In [ ]:
reg=LinearRegression()
reg.fit(X_train,y_train)
y_pred=reg.predict(X_test)
print('R^2: ', r2(y_test,y_pred))
print('RMSE: ',math.sqrt(mse(y_test,y_pred)))


R^2:  0.5686849741586173
RMSE:  2534.9788528028785
